<a href="https://colab.research.google.com/github/sgerloff/fashion_one_shot_test/blob/main/google_colab/triplet_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The data is stored on google drive. You need to put the proper zip file into your own google drive, or link to mine.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


After connecting to google drive, we read the data, unzip it. To this end we import the git-repository and use scripts from there:

In [2]:
!rm -rf /content/fashion_one_shot_test
!git clone https://github.com/sgerloff/fashion_one_shot_test.git
!chmod +x /content/fashion_one_shot_test/google_colab_utility/*.sh
!/content/fashion_one_shot_test/google_colab_utility/setup_train_data.sh

Streaming output truncated to the last 5000 lines.
  inflating: train/image_cat1/014774_1186.jpg  
  inflating: train/image_cat1/014761_1185.jpg  
  inflating: train/image_cat1/014729_1181.jpg  
  inflating: train/image_cat1/014810_1188.jpg  
  inflating: train/image_cat1/014720_1181.jpg  
  inflating: train/image_cat1/014744_1183.jpg  
  inflating: train/image_cat1/014871_1195.jpg  
  inflating: train/image_cat1/014713_1181.jpg  
  inflating: train/image_cat1/014859_1195.jpg  
  inflating: train/image_cat1/014870_1195.jpg  
  inflating: train/image_cat1/014706_1180.jpg  
  inflating: train/image_cat1/014711_1180.jpg  
  inflating: train/image_cat1/014714_1181.jpg  
  inflating: train/image_cat1/014857_1195.jpg  
  inflating: train/image_cat1/014863_1195.jpg  
  inflating: train/image_cat1/014737_1181.jpg  
  inflating: train/image_cat1/014806_1188.jpg  
  inflating: train/image_cat1/014721_1181.jpg  
  inflating: train/image_cat1/014804_1188.jpg  
  inflating: train/image_cat1/014767_

We  can import scripts and python modules from our repository in the following fashion:

In [3]:
from fashion_one_shot_test.google_colab_utility.get_train_database import get_clean_database

In [4]:
df = get_clean_database("/content/drive/MyDrive/deepfashion2/category_id_1_deepfashion_train.joblib", min_counts=20)

Contains 46457 entries with less then 20 pairs, which we will drop.


In [5]:
import os
image_folder = "train/image_cat1"
df["image"] = df["image"].apply(lambda x: os.path.join(image_folder, x.split("\\")[-1]))

Now lets gather all the components and train some models:

In [6]:
from fashion_one_shot_test.triplet_batch import TripletBatch
from fashion_one_shot_test.models.triplet_model import TripletModel
from fashion_one_shot_test.metrics.triplet_metrics import PessimisticTripletMetric
import tensorflow_addons as tfa
import tensorflow as tf

In [31]:
resolution=224
triplet_batch = TripletBatch(df, resolution)
training_size = df["pair_id"].nunique() // 20
bs = 32
dataset = tf.data.Dataset.from_generator(triplet_batch.tf_generator, args=[bs, training_size],
                                         output_types=(tf.float16, tf.float16),
                                         output_shapes=([resolution, resolution, 3], ())
                                         )

dataset = dataset.batch(bs, drop_remainder=True).repeat().prefetch(2)

metric = PessimisticTripletMetric()

model = TripletModel((resolution, resolution,3), trainable=True)
model.compile(loss=tfa.losses.TripletSemiHardLoss(margin=1.0), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005), 
              metrics=[metric.score])

#Optional
# model = tf.keras.models.load_model(f"triplet_deepfashion_{datetime.datetime.now().strftime('%m/%d/%Y')}.h5",
#                                    custom_objects={"score": metric.score})

model.fit(dataset, steps_per_epoch=training_size, epochs=100)

Epoch 1/100
45/45 [==============================] - 25s 426ms/step - loss: 0.9288 - score: 0.0000e+00
Epoch 2/100
45/45 [==============================] - 19s 427ms/step - loss: 0.7235 - score: 0.0304
Epoch 3/100
45/45 [==============================] - 19s 428ms/step - loss: 0.5431 - score: 0.0148
Epoch 4/100
45/45 [==============================] - 19s 430ms/step - loss: 0.3841 - score: 0.0678
Epoch 5/100
45/45 [==============================] - 19s 430ms/step - loss: 0.2798 - score: 0.0741
Epoch 6/100
45/45 [==============================] - 19s 431ms/step - loss: 0.1997 - score: 0.1765
Epoch 7/100
45/45 [==============================] - 19s 430ms/step - loss: 0.1702 - score: 0.2059
Epoch 8/100
45/45 [==============================] - 19s 433ms/step - loss: 0.1074 - score: 0.4532
Epoch 9/100
45/45 [==============================] - 19s 433ms/step - loss: 0.0943 - score: 0.5210
Epoch 10/100
45/45 [==============================] - 19s 430ms/step - loss: 0.0596 - score: 0.4459
Epoch

Don't forget to save your model and copy it to somewhere safe.

In [32]:
import datetime
filename = f"triplet_deepfashion_small_{datetime.datetime.now().strftime('%m/%d/%Y')}.h5"
model.save(filename, save_format="h5")
!(cp $filename /content/drive/MyDrive/deepfashion2/)